In [1]:
# Autor: Daniel Pinto
# Introduccion a la notacion asintotica parte 2
# Fecha: 2021/09/23 YYYY/MM/DD
from typing import List, TypeVar, Tuple, Any, Callable
from hypothesis import given, strategies as st
from IPython.display import Markdown, display
from itertools import accumulate
from functools import reduce

def display_(s : str) -> None:
    '''
    A way to display strings with markdown 
    in jupyter.
    '''
    display(
        Markdown(s)
    )


SUCCESS_COLOR = '#4BB543'
ERROR_COLOR   = '#B00020'

def color_text(s : str, color : str =SUCCESS_COLOR ) -> str:
    return f"<span style='color:{color}'> {s} </span>."


a      = TypeVar('a')
b      = TypeVar('b')
c      = TypeVar('c')

# Recursividad, El teorema maestro y Divide and Conquer

El principio de Divide and Conquer es bastante elegante: Dado un problema de tamaño $n$, lo podemos dividir en $a$ subproblemas de tamaño $\frac{n}{b}$ cada uno, y luego combinamos las soluciones usando algun otro algoritmo con complejidad $f(n)$.


Esto es mejor ejemplificado usando un ejercicio:

# Ejercicio: Maximum Subarray Sum

Dado un arreglo, dar el sub arreglo cuya suma es maxima.

```
Input: arr = [−2, 1, −3, 4, −1, 2, 1, −5, 4]
Output: (3,6)
Output:  puesto que: arr[3] + arr[4] + arr[5] + arr[6] = 6 y es de suma maxima.
```

In [ ]:

# [a,mid,b]
# [a]
# [a,mid]   pertenece exclusivamente a la parte izquierda
# [mid+1,b] pertenece exclusivamente a la parte derecha
# pasa por la mitad, a1,b1: [a..a1,mid,b1..b]
def _max_subarray_sum(arr : List[float], interval : Tuple[int,int]) -> Tuple[float,int,int] :
    (a,b) = interval
    # Lista de un solo elemento o vacia
    if a>=b:
        try:
            return (arr[a],a,a)
        except:
            return (0,-1,-1)
    # Lista de dos elementos [a,b]
    if a+1==b:
        # se retorna el maximo entre:
        # a+b si ambos son positivos
        # a   si b es mas negativo que a
        # b   si a es mas negativo que b 
        return max(
            [(arr[a] + arr[b],a,b)
            , (arr[a],a,a)
            , (arr[b],b,b)
            ]
            )

    # Sacamos el punto medio usando division entera
    mid : int = (a+b) // 2
    # Paso recursivo: obtenemos el valor maximo del arreglo
    # izquierdo y derecho que no atraviesan el centro
    (maxL,maxL_a,maxL_b) = _max_subarray_sum(arr,(a,mid))
    (maxR,maxR_a,maxR_b) = _max_subarray_sum(arr,(mid,b))

    # Para los que atraviesan el centro, usaremos esta funcion:
    def get_max_val_interval(sub : List[float]) -> Tuple[float,int]:
        # si [1,2,-5,4] es el arreglo del centro retorna:
        # max : [(0,0),(1,1),(3,2),(-2,3),(2,4)] = (3,2)
        # es decir, la suma maxima es 3 y empieza en el indice 2
        return max(
            accumulate(
                sub,
                func=lambda x_i,val: (x_i[0]+val,x_i[1]+1),
                initial=(0,0)
            )
        )

        # 
        # foldl (+) 0 [1..3] = [0,1,3,5]
        # scanl (+) 0 [1..3] = [0,1,3,5]
    
    # Obtenemos la suma maxima e indice del arreglo que esta a la derecha
    # de mid (sin incluirlo)
    (maxMR,_maxR_b) = get_max_val_interval(arr[mid+1:b+1])
    # Obtenemos el arreglo que esta a la izquierda de mid (sin incluirlo)
    # necesitamos esto puesto que si a=0, entonces arr[mid:a-1:-1] = arr[mid:-1:-1] = []
    # y nosotros queremos obtener la lista de la izquiera, no una no vacia
    aux : List[float] = arr[mid-1:a-1:-1] if a != 0 else arr[mid-1::-1]
    # Obtenemos la suma maxima e indice del arreglo que esta a la derecha
    # de mid (sin incluirlo)
    (maxML,_maxL_a) = get_max_val_interval(aux)
    # fijemonos que tanto maxM_b como maxM_a (los indices), estan con respecto al arreglo interno:
    # es decir, si nuestro arreglo original es: [2,[1,2,5,4,7],5]
    # entonces: (maxMR,maxMR_b) = (11,1) porque 4+7=11 
    # por lo tanto necesitamos desplazar el indice a la posicion: mid+indice en el caso
    # de la derecha y mid-indice en el caso de la izquierda
    maxM_b         = _maxR_b + mid
    maxM_a         = mid - _maxL_a 
    # el maximo valor que pasa por el centro
    # es la suma del maximo valor de derecha + izquierda + la pieza del centro
    maxM           = maxMR + maxML + arr[mid]
    
    # finalmente, retornamos  el valor maximo
    if maxM == max([maxL,maxR,maxM]):
        return (maxM,maxM_a, maxM_b)
    if maxL == max([maxL,maxR,maxM]):
        return (maxL,maxL_a,maxL_b)
    
    return (maxR,maxR_a,maxR_b)


def max_subarray_sum(arr : List[float]) -> Tuple[float,int,int]:
    return _max_subarray_sum(arr,(0,len(arr)-1))

def sum_array_test(f : Callable[[List[float]],Tuple[float,int,int]]) -> None:
    @given(ys = st.lists(st.integers(min_value=-20,max_value=20),max_size=20),)
    def _sum_array_test(ys : Any) -> None:
        xs : List[float] = list(ys)
        n : int = len(xs)
        indexes : List[Tuple[int,int]] = [(x,y) for x in range(n) for y in range(x,n)]

        try:
            (maximum_sum,index) = max(map(lambda a_b: (sum(xs[a_b[0]:a_b[1]+1]),a_b), indexes))
        except:
            (maximum_sum,index) = (0,(-1,-1))

        (val,i,j) = f(xs)

        tol = 10**(-10)

        try:
            assert(abs(val-maximum_sum) < tol)
        except Exception:
            display_(
                f"{color_text('Test Failed!',color=ERROR_COLOR)}<br>" +
                f"The maximum sum was: `{maximum_sum}` <br>" + 
                f"Got: `{val}` <br>"  +
                f"Array: <br>" +
                f"`{xs}`<br>" + 
                f"Solution indexes: `{index}` + <br>" + 
                f"Got: `{(i,j)}`"
            )
            return
        
        display_(color_text("Test successful!"))

    _sum_array_test()


arr : List[float] = [-2, 1, -3, 4, -1, 2, 1, -5, 4]
(val,a,b)         = max_subarray_sum(arr)
print((val,a,b))
sum_array_test(max_subarray_sum)



Fijemonos que el algoritmo general el siguiente arbol de recursion:

<center>

![max subarray sum recursion tree](./img/arbol_recursivo.png)

</center>

Cada nodo tiene exactamente $a=2$ hijos, los cuales trabajan con $\frac{n}{b=2}$ de los datos a la vez, ademas, notemos que para combinar los resultados, hacemos: $f=O(n)$ operaciones: encontrar el subarreglo maximo de $[a,mid)$, encontrar el subarreglo maximo $(mid,b]$, para finalmente unirlos $[a,b]$ y retornar el maximo entre este y los resultados recursivos.

Esto sugiere que debemos recorrer todo el arbol, y que el costo de traslado de cada nodo es $O\left(\dfrac{N}{2^{nivel}} \right)$, por lo tanto, nuestra complejidad sera:

$$
O\left(\sum_{0 \leq i \leq log_2(N)}  (a=2^i) \cdot \dfrac{f=N}{b=2^i} \right) = O\left(\sum_{0 \leq i \leq log_2(N)} N \right) = O(N \ log(n))
$$

# Teorema maestro

<center>

![teorema maestro](./img/teorema_maestro.png)

</center>


En el caso anterior, notemos que:


$$
f(n) = O(n) = O(n \cdot 1) = O(n \cdot log^0(n)) \implies k = 0
$$

Entonces, por el caso 2 del teorema maestro, la complejidad sera:

$$
O(n^{log_2(2)} \cdot log^{0+1}n) = O(n \ log(n))
$$
